In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pulp
import sys
import os
import importlib.util
import importlib
import gurobipy as gp

# モジュールを強制的に再ロード（古いキャッシュをクリア）
if 'code' in sys.modules:
    del sys.modules['code']
# サブモジュールも削除
for module_name in list(sys.modules.keys()):
    if module_name.startswith('code.'):
        del sys.modules[module_name]

current_dir = os.getcwd()
spec = importlib.util.spec_from_file_location("code", os.path.join(current_dir, "code", "__init__.py"))
code_module = importlib.util.module_from_spec(spec)
sys.modules["code"] = code_module
spec.loader.exec_module(code_module)

# サブモジュールも再ロード
importlib.reload(code_module)

make_tensor_grid_2d = code_module.make_tensor_grid_2d
make_tensor_grid_3d = code_module.make_tensor_grid_3d
product_beta_density = code_module.product_beta_density
solve_mechanism_symmetry_2agents_with_synergy = code_module.solve_mechanism_symmetry_2agents_with_synergy
solve_mechanism_symmetry_2agents_with_synergy_iterative = code_module.solve_mechanism_symmetry_2agents_with_synergy_iterative
save_results_symmetry_2agents_with_synergy = code_module.save_results_symmetry_2agents_with_synergy
load_results_symmetry_2agents_with_synergy = code_module.load_results_symmetry_2agents_with_synergy

print("モジュールを再ロードしました")


In [ ]:
home_license = os.path.expanduser('~/gurobi.lic')
os.environ['GRB_LICENSE_FILE'] = home_license

SOLVER = pulp.GUROBI(msg=True)


# ケース1: 2人2財1シナジー（対称性制約あり）


In [ ]:
# ケース1: 財a, 財b, シナジーαはBeta(1,1)
# 対称性制約のため、参加者1と参加者2は同じpointsとweightsを使用
NX, NY, NZ = 15, 15, 15
BETA_PARAMS = [
    (1.0, 1.0),
    (1.0, 1.0),
    (1.0, 1.0),
]

points1, weights1 = make_tensor_grid_3d(NX, NY, NZ, BETA_PARAMS)
points2, weights2 = points1.copy(), weights1.copy()

print(f"#types agent1 = {len(points1)}")
print(f"#types agent2 = {len(points2)}")
print(f"Total type combinations = {len(points1) * len(points2)}")
print(f"First point agent1: {points1[0]}, weight: {weights1[0]}")
print(f"First point agent2: {points2[0]}, weight: {weights2[0]}")
print(f"Points are equal: {points1 == points2}")
print(f"Weights are equal: {weights1 == weights2}")


In [ ]:
status, obj_val, u1_sol, u2_sol, p1_sol, p2_sol, n_iter = solve_mechanism_symmetry_2agents_with_synergy_iterative(
    points1, weights1, (NX, NY, NZ),
    points2, weights2, (NX, NY, NZ),
    solver=SOLVER
)

print("LP status:", status)
print("Optimal revenue:", obj_val)
print(f"Number of iterations: {n_iter}")

# 対称性の確認
print(f"\n対称性の確認:")
print(f"u1 is u2 (same reference): {u1_sol is u2_sol}")
print(f"p1 is p2 (same reference): {p1_sol is p2_sol}")
print(f"u1 == u2 (values): {np.allclose(u1_sol, u2_sol)}")
print(f"p1 == p2 (values): {np.allclose(p1_sol, p2_sol)}")

# 結果を保存
filepath_case1 = save_results_symmetry_2agents_with_synergy(
    points1, weights1, points2, weights2,
    u1_sol, u2_sol, p1_sol, p2_sol,
    obj_val, status,
    grid_sizes1=(NX, NY, NZ),
    grid_sizes2=(NX, NY, NZ),
    n_iter=n_iter,
    filename="results_symmetry_synergy_case1.npz"
)


# ケース2: 財a, 財bはBeta(1,1)、シナジーαはU[-1,0]


In [ ]:
NX2, NY2, NZ2 = 15, 15, 15
BETA_PARAMS_MIXED = [
    {'type': 'beta', 'params': (1.0, 1.0), 'range': (0, 1)},
    {'type': 'beta', 'params': (1.0, 1.0), 'range': (0, 1)},
    {'type': 'uniform', 'range': (-1, 0)},
]

points1_2, weights1_2 = make_tensor_grid_3d(NX2, NY2, NZ2, BETA_PARAMS_MIXED)
points2_2, weights2_2 = points1_2.copy(), weights1_2.copy()

print(f"#types agent1 = {len(points1_2)}")
print(f"#types agent2 = {len(points2_2)}")
print(f"Total type combinations = {len(points1_2) * len(points2_2)}")
print(f"ex. first point agent1, weight = {points1_2[0]}, {weights1_2[0]}")
print(f"ex. first point agent2, weight = {points2_2[0]}, {weights2_2[0]}")


In [ ]:
status2, obj_val2, u1_sol2, u2_sol2, p1_sol2, p2_sol2, n_iter2 = solve_mechanism_symmetry_2agents_with_synergy_iterative(
    points1_2, weights1_2, (NX2, NY2, NZ2),
    points2_2, weights2_2, (NX2, NY2, NZ2),
    solver=SOLVER
)

print("LP status:", status2)
print("Optimal revenue:", obj_val2)
print(f"Number of iterations: {n_iter2}")

# 結果を保存
filepath_case2 = save_results_symmetry_2agents_with_synergy(
    points1_2, weights1_2, points2_2, weights2_2,
    u1_sol2, u2_sol2, p1_sol2, p2_sol2,
    obj_val2, status2,
    grid_sizes1=(NX2, NY2, NZ2),
    grid_sizes2=(NX2, NY2, NZ2),
    n_iter=n_iter2,
    filename="results_symmetry_synergy_case2.npz"
)


# ケース3: 財a, 財bはBeta(1,1)、シナジーαはU[-1,1]


In [ ]:
NX3, NY3, NZ3 = 15, 15, 20
BETA_PARAMS_MIXED3 = [
    {'type': 'beta', 'params': (1.0, 1.0), 'range': (0, 1)},
    {'type': 'beta', 'params': (1.0, 1.0), 'range': (0, 1)},
    {'type': 'uniform', 'range': (-1, 1)},
]

points1_3, weights1_3 = make_tensor_grid_3d(NX3, NY3, NZ3, BETA_PARAMS_MIXED3)
points2_3, weights2_3 = points1_3.copy(), weights1_3.copy()

print(f"#types agent1 = {len(points1_3)}")
print(f"#types agent2 = {len(points2_3)}")
print(f"Total type combinations = {len(points1_3) * len(points2_3)}")
print(f"ex. first point agent1, weight = {points1_3[0]}, {weights1_3[0]}")


In [ ]:
status3, obj_val3, u1_sol3, u2_sol3, p1_sol3, p2_sol3, n_iter3 = solve_mechanism_symmetry_2agents_with_synergy_iterative(
    points1_3, weights1_3, (NX3, NY3, NZ3),
    points2_3, weights2_3, (NX3, NY3, NZ3),
    solver=SOLVER
)

print("LP status:", status3)
print("Optimal revenue:", obj_val3)
print(f"Number of iterations: {n_iter3}")

# 結果を保存
filepath_case3 = save_results_symmetry_2agents_with_synergy(
    points1_3, weights1_3, points2_3, weights2_3,
    u1_sol3, u2_sol3, p1_sol3, p2_sol3,
    obj_val3, status3,
    grid_sizes1=(NX3, NY3, NZ3),
    grid_sizes2=(NX3, NY3, NZ3),
    n_iter=n_iter3,
    filename="results_symmetry_synergy_case3.npz"
)
